<a href="https://colab.research.google.com/github/junginkim23/ds-section1-sprint2/blob/master/ai03_sc12x_%EA%B9%80%EC%A0%95%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

# SC12x

## Statistical Analysis

## 개요

여러분은 코스 수료를 마치고 CodeStates에 DS CA (Coaching  Assistant)로 합류했습니다.

여러분에게 주어진 메인 태스크는 세션 및 수강생 핸들링이지만. 

시간을 짜내어 사이드 프로젝트를 하고 싶어졌습니다.

AIB 팀원들과의 커피타임을 통해 수강생 데이터를 하나 받을 수 있었습니다.

> 기수별 학생들 성취도 데이터인데, 어떤 의미가 있는지 확인해주세요.

### Data Description

- F1 : `N`, `D`
- F2 : `Y`, `N`
- F3 : `A`, `B`, `C`, `D` (기수)
- F4 : Continuous
- F5 : Continuous
- F6 : `A` ~ `H`
- F7 : Continuous
- F8 : `X`, `Y`, `Z`
- F9 : `Y`, `N`

## 1. 다음 링크를 참조하여 데이터를 불러오세요. 

<https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/sc12x/csdata.csv>

이후 간단한 전처리를 하세요.


In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/sc12x/csdata.csv",index_col=0)

df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9
1,N,N,A,2.239542,0.773708,H,3.0,Y,Y
2,D,N,A,1.608900,0.133612,H,3.0,X,Y
3,N,N,A,2.544409,0.721873,H,8.0,Y,Y
4,N,Y,A,0.782857,0.259751,F,9.0,Y,N
5,N,N,A,1.532721,0.283534,E,4.0,Z,Y


In [5]:
df.describe()

,F4,F5,F7
count,200.000000,170.000000,197.00000
mean,2.512655,0.500227,4.93401
std,1.166105,0.289515,2.00273
min,0.022079,0.010212,0.00000
25%,1.732794,0.244191,3.00000
50%,2.546567,0.486618,5.00000
75%,3.311934,0.753421,6.00000
max,5.868967,0.993412,11.00000


In [6]:
#null 값 확인 -> F2, F5, F7 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F1      200 non-null    object 
 1   F2      191 non-null    object 
 2   F3      200 non-null    object 
 3   F4      200 non-null    float64
 4   F5      170 non-null    float64
 5   F6      200 non-null    object 
 6   F7      197 non-null    float64
 7   F8      200 non-null    object 
 8   F9      200 non-null    object 
dtypes: float64(3), object(6)
memory usage: 15.6+ KB


In [8]:
print(df.F1.unique())
print(df.F2.unique()) 
print(df.F3.unique()) #F3에 소문자를 대문자로 변환해야함.
print(df.F6.unique())
print(df.F8.unique())
print(df.F9.unique())

['N' 'D']
['N' 'Y' nan]
['A' 'a' 'B' 'b' 'C' 'c' 'D' 'd']
['H' 'F' 'E' 'C' 'G' 'D' 'B']
['Y' 'X' 'Z']
['Y' 'N']


In [24]:
#결측치 제거 F2 & F7 
df = df[~(df['F2'].isnull())]
df = df[~(df['F7'].isnull())]

#F3 소문자 -> 대문자 변환
df['F3'] = df['F3'].str.upper()

#F5 결측값 대체 
df['F5'] = df['F5'].fillna(df['F5'].mean())

In [25]:
df.isnull().sum()

F1    0
F2    0
F3    0
F4    0
F5    0
F6    0
F7    0
F8    0
F9    0
dtype: int64

## 2. **F3**에 따라서 다른 Feature들이 어떤 경향을 나타내는지 해석해보세요.

- categorical feature의 경우 crosstab을 사용하여 [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)과 chi-square를 시도해 볼 수 있습니다.

- continuous feature의 경우 여러 시각화와 t-test를 할 수 있을 겁니다.

- 이를 바탕으로 **가장 유의미한** 그리고 **가장 무의미한** feature를 찾아보세요.

여기서 여러분들의 목표는 

`F3`에 따라서 다른 feature들이 영향이 있는지 없는지를 알아보는 

**가설검정** 입니다.

따라서 데이터 탐색 및 시각화 작업에 너무 많은 시간을 쏟지 마세요. 이것은 사이드 프로젝트 입니다.

In [34]:
# F3 & F1 2 sample chi_square test
# H0: F1과 F3은 서로 관계가 없다. (독립)
# H1: F1과 F3은 서러 관계가 있다. (종속)

from scipy.stats import chi2_contingency
import numpy as np

obs = pd. crosstab(df['F3'],df['F1'])


chi2_contingency(obs)
#p_value가 0.697로 0.05보다 크므로 귀무가설을 기각할 수 없다. 
#즉, F1과 F3는 서로 관계가 없다. 독립! 

(1.4377719467233523, 0.6967051489258402, 3, array([[ 8.68085106, 39.31914894],
        [ 8.31914894, 37.68085106],
        [ 8.5       , 38.5       ],
        [ 8.5       , 38.5       ]]))

In [40]:
# F3 & F6 2 sample chi_square test
# H0: F6과 F3은 서로 관계가 없다. (독립)
# H1: F6과 F3은 서러 관계가 있다. (종속)

from scipy.stats import chi2_contingency
import numpy as np

obs = pd. crosstab(df['F3'],df['F6'])

chi2_contingency(obs)
#p_value가 0.692로 0.05보다 크므로 귀무가설을 기각할 수 없다. 
#즉, F6과 F3는 서로 관계가 없다. 독립! 

(14.558616404926772,
 0.6920174637926277,
 18,
 array([[ 0.25531915,  1.27659574,  1.53191489,  4.59574468,  9.19148936,
         10.9787234 , 20.17021277],
        [ 0.24468085,  1.22340426,  1.46808511,  4.40425532,  8.80851064,
         10.5212766 , 19.32978723],
        [ 0.25      ,  1.25      ,  1.5       ,  4.5       ,  9.        ,
         10.75      , 19.75      ],
        [ 0.25      ,  1.25      ,  1.5       ,  4.5       ,  9.        ,
         10.75      , 19.75      ]]))

In [ ]:
# F3 & F2 2 sample chi_square test
# H0: F2과 F3은 서로 관계가 없다. (독립)
# H1: F2과 F3은 서러 관계가 있다. (종속)

from scipy.stats import chi2_contingency
import numpy as np

obs = pd. crosstab(df['F3'],df['F2'])

chi2_contingency(obs)
#p_value가 0.697로 0.05보다 크므로 귀무가설을 기각할 수 없다. 
#즉, F2과 F3는 서로 관계가 없다. 독립! 


In [41]:
# F3 & F8 2 sample chi_square test
# H0: F8과 F3은 서로 관계가 없다. (독립)
# H1: F8과 F3은 서러 관계가 있다. (종속)

from scipy.stats import chi2_contingency
import numpy as np

obs = pd. crosstab(df['F3'],df['F8'])

chi2_contingency(obs)
#p_value가 0.541로 0.05보다 크므로 귀무가설을 기각할 수 없다. 
#즉, F8과 F3는 서로 관계가 없다. 독립! 

(5.021174328115594,
 0.5411002323035743,
 6,
 array([[ 9.95744681, 24.25531915, 13.78723404],
        [ 9.54255319, 23.24468085, 13.21276596],
        [ 9.75      , 23.75      , 13.5       ],
        [ 9.75      , 23.75      , 13.5       ]]))

In [42]:
# F3 & F9 2 sample chi_square test
# H0: F9과 F3은 서로 관계가 없다. (독립)
# H1: F9과 F3은 서러 관계가 있다. (종속)

from scipy.stats import chi2_contingency
import numpy as np

obs = pd. crosstab(df['F3'],df['F9'])

chi2_contingency(obs)
#p_value가 0.904로 0.05보다 크므로 귀무가설을 기각할 수 없다. 
#즉, F9과 F3는 서로 관계가 없다. 독립! 

(0.5646820912389221, 0.904467812423293, 3, array([[26.04255319, 21.95744681],
        [24.95744681, 21.04255319],
        [25.5       , 21.5       ],
        [25.5       , 21.5       ]]))

In [44]:
#F3 & F4 ANOVA test f_oneway
#H0: 기수별로(A,B,C,D) F4값의 평균의 차이가 없다.
#H1: 기수별로(A,B,C,D) F4값의 평균의 차이가 없는 것은 아니다. 

F4_A = df[df['F3']=='A'][['F4']]
F4_B = df[df['F3']=='B'][['F4']]
F4_C = df[df['F3']=='C'][['F4']]
F4_D = df[df['F3']=='D'][['F4']]

from scipy.stats import f_oneway
f_oneway(F4_A,F4_B,F4_C,F4_D)

# p_value가 0.05보다 작으므로 귀무가설을 기각한다. 즉 F4의 평균이 같다고 할 수 없다.

F_onewayResult(statistic=array([23.41731437]), pvalue=array([6.94588824e-13]))

In [45]:
#F3 & F5 ANOVA test f_oneway
#H0: 기수별로(A,B,C,D) F5값의 평균의 차이가 없다.
#H1: 기수별로(A,B,C,D) F5값의 평균의 차이가 없는 것은 아니다. 

F5_A = df[df['F3']=='A'][['F5']]
F5_B = df[df['F3']=='B'][['F5']]
F5_C = df[df['F3']=='C'][['F5']]
F5_D = df[df['F3']=='D'][['F5']]

from scipy.stats import f_oneway
f_oneway(F5_A,F5_B,F5_C,F5_D)

# p_value가 0.05보다 크므로 귀무가설을 기각할 수 없다. 즉 F5의 평균이 차이가 거의 없다고 할 수 있다.

F_onewayResult(statistic=array([0.19930143]), pvalue=array([0.89676356]))

In [47]:
# F3 & F7 2 sample chi_square test
# H0: F7과 F3은 서로 관계가 없다. (독립)
# H1: F7과 F3은 서러 관계가 있다. (종속)

from scipy.stats import chi2_contingency
import numpy as np

obs = pd. crosstab(df['F3'],df['F7'])

chi2_contingency(obs)
#p_value가 0.696로 0.05보다 크므로 귀무가설을 기각할 수 없다. 
#즉, F7과 F3는 서로 관계가 없다. 독립! 

(1.4377719467233523, 0.6967051489258402, 3, array([[ 8.68085106, 39.31914894],
        [ 8.31914894, 37.68085106],
        [ 8.5       , 38.5       ],
        [ 8.5       , 38.5       ]]))

## 3. 분석한 내용들을 설명하는 리포트를 작성해보세요. 

분석의 끝은 결국 설명입니다.

통계 분석에 대한 이해도가 적은 다른 크루들이 이해 할 수 있도록 

분석한 내용들을 설명하는 리포트를 작성해보세요. 

필요한 경우 코드 / 텍스트 내용을 더 추가해도 좋습니다.

### Description

F3를 기준으로 나머지 feature와의 관계를 chi_square test와 ANOVA test로 파악해보았습니다. 

- chi_square test (F1,F2,F6,F7,F8,F9)
카이제곱테스트를 시행한 결과 F3와 해당 feature와는 모두 독립의 관계임을 밝힐 수 있었다. 

- ANOVA test (F4, F5)
아노바테스트를 시행한 결과 F4는 p_value가 0.05보다 작은 수치가 나와 귀무가설을 기각하였고 F4의 평균과 같다고 할 수 없다라는 결론을 도출할 수 있었다. 반면에 F5는 0.05보다 큰 수치가 나왔기 때문에 귀무가설을 기각할 수 없었다.

## SC 3점 요구사항:

위에서 요구된 사항들을 모두 정확하게 만족하고, 시간이 남았다면 아래 내용을 충족하는 경우 3점을 얻을 수 있습니다. 

아래 요구사항들 중 하나의 요구사항만 충족하면 되며, 모두 **optional** 임을 다시 한 번 강조합니다. (필수 요구사항이 아닙니다.)

- Numerical 데이터에 대해서 normalize를 한 다음, 위에서 진행 했었던 분석을 다시 시행해보세요. (F4, F5, F7 vs F3)

- Categorical 데이터를 one-hot encoding을 통해서 numerical 로 변환 후, 위에서 진행 했었던 분석을 다시 시행해보세요. (F1, F2, F6, F8, F9 vs F3)

In [ ]:
#Numerical 데이터에 대해서 normalize하라는 것이 numeric->category하라는 것으로 생각됩니다. 이러한 과정은 이미 위에서 진행되었기에 생략하였습니다.

#categorical -> numerical using one_hot_encoding 
new_result = []

for feature in ['F1','F2','F6','F8','F9']:
   new_df = pd.get_dummies(data = df[feature], columns = [feature], prefix = feature)
   for new_feature in new_df.columns:
     obs = pd.crosstab(df['F3'],new_df[new_feature])
     new_result.append((new_feature,chi2_contingency(obs, correction = False)[1],chi2_contingency(obs, correction = False)[2]))

new_result

[('F1_D', 0.545872281592922, 3),
 ('F1_N', 0.545872281592922, 3),
 ('F2_N', 0.4654547449849822, 3),
 ('F2_Y', 0.3502796549106317, 3),
 ('F6_C', 0.3193693920911738, 3),
 ('F6_D', 0.8910742882211841, 3),
 ('F6_E', 0.8359563426104062, 3),
 ('F6_F', 0.06086603526800449, 3),
 ('F6_G', 0.7301292180920782, 3),
 ('F6_H', 0.50437053784125, 3),
 ('F8_X', 0.19831409775857844, 3),
 ('F8_Y', 0.8515755888890483, 3),
 ('F8_Z', 0.6631031348015854, 3),
 ('F9_N', 0.8792640844352739, 3),
 ('F9_Y', 0.8792640844352739, 3)]